In [1]:
import numpy as np
from collections import defaultdict
from load_data import load_process_essays

import logging
import datetime
import pandas as pd

## Load and Pre-Process Essays

In [2]:
import pickle
from CrossValidation import cross_validation
from BrattEssay import load_bratt_essays
from IterableFP import flatten
from Settings import Settings
from window_based_tagger_config import get_config

CV_FOLDS = 5
DEV_SPLIT = 0.1

settings = Settings()
data_set = "SkinCancer"
# data_set = "CoralBleaching"

root_folder = settings.data_directory + data_set + "/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"
training_pickled = settings.data_directory + data_set + "/Thesis_Dataset/training.pl"
models_folder = root_folder + "Models/Bi-LSTM/"

config = get_config(training_folder)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [3]:
%%time
config = get_config(training_folder)
train_essays = load_process_essays(**config)

870 files found
870 essays processed
CPU times: user 29.2 s, sys: 425 ms, total: 29.7 s
Wall time: 35.2 s


In [4]:
%%time
config = get_config(test_folder)
test_essays = load_process_essays(**config)

218 files found
218 essays processed
CPU times: user 7.58 s, sys: 82.2 ms, total: 7.66 s
Wall time: 8.27 s


## Prepare Tags

In [5]:
def get_essay_stats(essays, lbl):
    tag_freq = defaultdict(int)
    unique_words = set()
    num_wds = 0
    num_sents = 0
    
    for essay in essays:
        for sentence in essay.sentences:
            num_sents += 1
            for word, tags in sentence:
                num_wds += 1
                unique_words.add(word)
    return {
        "dataset": lbl,
        "num_essays": len(essays),
        "num_sents": num_sents,
        "num_words": num_wds,
        "vocab": len(unique_words)        
    }

In [6]:
all_essays = train_essays + test_essays

a = get_essay_stats(train_essays, "train")
b = get_essay_stats(test_essays,  "test")
c = get_essay_stats(all_essays,    "all")

pd.DataFrame([a,b,c])

,dataset,num_essays,num_sents,num_words,vocab
0,train,870,8573,145471,1582
1,test,218,2097,35402,899
2,all,1088,10670,180873,1623


In [7]:
# CB

# dataset	    num_essys	num_sents	num_words	vocab
# 0	train	    901     	8280    	136948  	1641
# 1	test	    226     	1918    	30699   	927
# 2	all     	1127    	10198   	167647  	1677

In [8]:
# SC

# dataset	num_essays	num_sents	num_words	vocab
# 0	train	870     	8573    	145471  	1582
# 1	test	218     	2097    	35402   	899
# 2	all 	1088    	10670   	180873  	1623